### Queue

In [1]:
from queue import deque # from collections import deque # both are the same

Q = deque([1,2,3])
print( len(Q) )

Q.append(5)
print( Q )

Q.appendleft(6) # not used for normal Q
print( Q )

x = Q.pop() # not used for normal Q
print(x, Q)

x = Q.popleft()
print(x, Q)

3
deque([1, 2, 3, 5])
deque([6, 1, 2, 3, 5])
5 deque([6, 1, 2, 3])
6 deque([1, 2, 3])


In [2]:
# Queue: implemented by circular array
# tail (append) and head (popleft) are same index initially -> empty: t==h; len=(h-t)%size; full: (h+1)%size==t # head always empty
class Queue:
    def __init__(self, size=8):
        self.L = [0]*size
        self.t = 0
        self.h = 0
    
    def __len__(self):
        return (self.h-self.t) % len(self.L)
    
    def append(self, x):
        if (self.h+1) % len(self.L) == self.t:
            print("failed because is full")
        else:
            self.L[self.h] = x
            self.h = (self.h+1) % len(self.L)
    
    def popleft(self):
        if self.t==self.h:
            print("failed becuase is empty")
        else:
            x = self.L[self.t]
            self.t = (self.t+1) % len(self.L)
            return x
        
    def head(self):
        return self.L[ (self.h-1) % len(self.L) ]
    
    def tail(self):
        return self.L[self.t]
        
Q = Queue(4)
Q.popleft() # fail
Q.append(0)
Q.append(1)
Q.append(2)
Q.append(3) # fail
Q.append(4) # fail
Q.popleft() # remove 0
Q.append(10)
Q.append(11) # fail
Q.popleft() # remove 1
print( len(Q), Q.head(), Q.tail() ) # 2, 10, 2

failed becuase is empty
failed because is full
failed because is full
failed because is full
2 10 2


### Tree

In [3]:
class Tree:
    def __init__(self, val=0, left=None, right=None):
        self.val, self.left, self.right = val, left, right
        
#     def height(self): # ugly due to the isLeaf-check that prevent from "none has no attribute height"
#         if self:
#             return 1 + max( self.left.height() if self.left else 0, self.right.height() if self.right else 0 )
#         else:
#             return 0
    
    def height(self):
        def f(T):
            return 1 + max( f(T.left), f(T.right) ) if T else 0
        return f(self)
    
    def vis(self):
        H = self.height()
        L = [ [-1]*2**i for i in range(H) ]
        def f(T, level=0, levelIdx=0):
            if T:
                L[level][levelIdx] = T.val
                f(T.left , level+1, 2*levelIdx)
                f(T.right, level+1, 2*levelIdx+1)
        f(self)
        return L
    
    def dlr(self):
        f = lambda T: [T.val] + f(T.left) + f(T.right) if T else []
        return f(self)
    
    def ldr(self):
        f = lambda T: f(T.left) + [T.val] + f(T.right) if T else []
        return f(self)
    
    def lrd(self):
        f = lambda T: f(T.left) + f(T.right) + [T.val] if T else []
        return f(self)
        
def fullExample(h):
    T = Tree(0) if h>0 else None
    def f(T, i=0):
        if 2*i+1 < 2**h-1:
            T.left  = Tree(2*i+1)
            f(T.left, 2*i+1)
            T.right = Tree(2*i+2)
            f(T.right, 2*i+2)
    f(T)
    return T
    
T = Tree(0, Tree(1, Tree(3), Tree(4)), Tree(2, Tree(5), Tree(6)))
print( T.height() )
print( T.vis() )
print( T.dlr() )
print( T.ldr() )
print( T.lrd() )

print("-"*10)
U0 = fullExample(4)
print( U0.vis() )

3
[[0], [1, 2], [3, 4, 5, 6]]
[0, 1, 3, 4, 2, 5, 6]
[3, 1, 4, 0, 5, 2, 6]
[3, 4, 1, 5, 6, 2, 0]
----------
[[0], [1, 2], [3, 4, 5, 6], [7, 8, 9, 10, 11, 12, 13, 14]]


### BST

In [5]:
class BST(Tree):
    def __init__(self, val=0, left=None, right=None):
        super().__init__(val, left, right)
           
    def insert(self, x):
        def f(T, x): # return tree that has inserted node
            if not T:
                T = BST(x)
            elif x<=T.val:
                T.left  = f(T.left, x)
            else:
                T.right = f(T.right, x)
            return T
        f(self, x)
    
    def delete(self, x):
        def f(T, x): # return tree that hasn't deleted node # parent pointer is NOT important
            if T:
                if x==T.val:
                    if not T.left: # leaf or right only
                        T = T.right
                    elif not T.right: # left only
                        T = T.left
                    else:
                        p = T.left # have both left and right -> find max of left subtree
                        while p.right:
                            p = p.right
                        T.val  = p.val
                        T.left = f(T.left, p.val)
                elif x<T.val:
                    T.left  = f(T.left, x)
                else:
                    T.right = f(T.right, x)
                return T
        f(self, x)
        
def fullBSTExample(h):
    T = BST( 2**(h-1)-1 ) if h>0 else None
    def f(T, l=0, r=2**h-2):
        if l<r:
            T.left  = BST( (l+T.val-1)//2 )
            f(T.left , l, T.val-1)
            T.right = BST( (T.val+1+r)//2 )
            f(T.right, T.val+1, r)
    f(T)
    return T

def isBST1(T, minv=-float('inf'), maxv=float('inf')):
    return (minv<T.val<maxv) and isBST1(T.left, minv, T.val) and isBST1(T.right, T.val, maxv) if T else True

def isBST2(T):
    ldr = lambda T: ldr(T.left)+[T.val]+ldr(T.right) if T else []
    L   = ldr(T)
    return sum([ L[i]<L[i+1] for i in range(len(L)-1) ])==len(L)-1
        
T = BST(3, BST(1, BST(3), BST(4)), BST(2, BST(5), BST(6)))
print( T.vis() )
U = fullBSTExample(4)
print( U.vis() )
print( isBST1(U) , isBST1(U) )
print( isBST1(U0), isBST2(U0) )

[[3], [1, 2], [3, 4, 5, 6]]
[[7], [3, 11], [1, 5, 9, 13], [0, 2, 4, 6, 8, 10, 12, 14]]
True True
False False


In [6]:
x = U.insert(4.5)
x = U.insert(2.7)
print( U.vis() )
print( U.ldr() )

print("-"*10)

U.delete(4.5)
print( U.vis() )
U.delete(2.7)
print( U.vis() )
U.delete(12)
print( U.vis() )
U.delete(3)
print( U.vis(), U.ldr() )
U.delete(1)
print( U.vis(), U.ldr() )

[[7], [3, 11], [1, 5, 9, 13], [0, 2, 4, 6, 8, 10, 12, 14], [-1, -1, -1, 2.7, -1, 4.5, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]]
[0, 1, 2, 2.7, 3, 4, 4.5, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
----------
[[7], [3, 11], [1, 5, 9, 13], [0, 2, 4, 6, 8, 10, 12, 14], [-1, -1, -1, 2.7, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]]
[[7], [3, 11], [1, 5, 9, 13], [0, 2, 4, 6, 8, 10, 12, 14]]
[[7], [3, 11], [1, 5, 9, 13], [0, 2, 4, 6, 8, 10, -1, 14]]
[[7], [2, 11], [1, 5, 9, 13], [0, -1, 4, 6, 8, 10, -1, 14]] [0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14]
[[7], [2, 11], [0, 5, 9, 13], [-1, -1, 4, 6, 8, 10, -1, 14]] [0, 2, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14]


### Heap

In [7]:
from math import log, ceil, floor
import random

# parent2child i->(2*i+1,2*i+2) | child2parent i->(i-1)//2
# l-th level: 2^l level nodes | m level nodes at log(m)-th level 
# h levels: 2^h -1 total nodes | n total nodes: ceil(log(n+1)) levels
# i-th node at floor( log(i+1) ) level; i-(2**level-1)-th level idx | l-level j-level-idx: 2^(l-1)+j -th node

class Heap(list):   
    def height(self):
        return ceil( log(len(self)+1)/log(2) )
    
    def vis(self):
        L = [ [-1]*2**i for i in range(self.height()) ]
        for i,x in enumerate(self):
            level = floor(log(i+1)/log(2))
            L[ level ][ i-(2**level-1) ] = x
        return L
    
    def insert(self,x): # overloading
        self.append(x)
        i = len(self)-1
        while i>0 and self[ (i-1)//2 ] > self[i]:
            self[ (i-1)//2 ], self[i] = self[i], self[ (i-1)//2 ]
            i = (i-1)//2
    
    def getMin(self):
        return self[0]
    
    def popMin(self):
        if len(self)==1:
            return self.pop()
        elif len(self)>1:
            target  = self[0]
            self[0] = self.pop()
            i = 0
            while True:
                compareL = [(self[i],"d")] + ([(self[2*i+1],"l")] if 2*i+1<len(self) else []) \
                    + ([(self[2*i+2],"r")] if 2*i+2<len(self) else [])
                _, minNode = min(compareL)
                if minNode=="d":
                    break
                if minNode=="l":
                    self[i], self[2*i+1] = self[2*i+1], self[i]
                    i = 2*i+1
                else:
                    self[i], self[2*i+2] = self[2*i+2], self[i]
                    i = 2*i+2
            return target
            
def isHeap(H, i=0):
    if not H:
        return True
    else:
        l, r, s = i*2+1, i*2+2, len(H)
        if l>=s and r>=s:
            return True
        elif r>=s:
            return H[i]<H[l] and isHeap(H,l)
        else:
            return H[i]<H[l] and H[i]<H[r] and isHeap(H,l) and isHeap(H,r)
        
def heapSort(n):
    R = list(range(n))
    random.shuffle(R)
    H = Heap()
    for ele in R:
        H.insert(ele)
    return [ H.popMin() for _ in range(len(R)) ]
        
H0 = Heap(list(range(10)))
H1 = Heap([1,7,4,9,14,5,6])
H2 = Heap([1,7,4,9,14,5,6,6])
print( isHeap(H0), H0.height(), H0.vis() )
print( isHeap(H1), H1.height(), H1.vis() )
print( isHeap(H2), H2.height(), H2.vis() )

True 4 [[0], [1, 2], [3, 4, 5, 6], [7, 8, 9, -1, -1, -1, -1, -1]]
True 3 [[1], [7, 4], [9, 14, 5, 6]]
False 4 [[1], [7, 4], [9, 14, 5, 6], [6, -1, -1, -1, -1, -1, -1, -1]]


In [8]:
H1.insert(8)
print( H1.vis() )
H1.popMin()
print( H1.vis() )

print( heapSort(20) )

[[1], [7, 4], [8, 14, 5, 6], [9, -1, -1, -1, -1, -1, -1, -1]]
[[4], [7, 5], [8, 14, 9, 6]]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


In [2]:
import heapq
H = [3,1,4,1,5,9,2,6]
heapq.heapify(H) # O(n)
print(H)
print(heapq.heappop(H))
print(H)
heapq.heappush(H, 2)
print(H)

[1, 1, 2, 3, 5, 9, 4, 6]
1
[1, 3, 2, 6, 5, 9, 4]
[1, 2, 2, 3, 5, 9, 4, 6]


### OrderedDict

In [60]:
# Review dict more
D = {"a":1, "b":2, "c":3}
print( D.popitem() )  # similar to list.pop() and set.pop()
print( D.pop("a") ) # similar to list.remove(x) and set.remove(x) and del D[x] (return is trivial) # not recommended due to confusion
print( D )
D.update({"d":5,"b":10}) # merge dict # same as D={**D,**E} # similar to set.update(set) and set.union(set) 
print(D)
# D.pop("a", 1) -> pop D['a'] if 'a' in D else 1
# D.get("a", 1) -> D['a'] if 'a' in D else 1
# Ordered dict is inherit from dict
# Ordered dict application: LRU cache
# Ordered dict is made by dict and Doubly Linked List

('c', 3)
1
{'b': 2}
{'b': 10, 'd': 5}


In [61]:
from collections import OrderedDict

In [62]:
O = OrderedDict() # same type as dict.items()
O['apple']  = 30
O['banana'] = 10
O['guava'] = 20
print(O, len(O), O.keys(), O.values(), O.items())
del O['banana']
print(O)

OrderedDict([('apple', 30), ('banana', 10), ('guava', 20)]) 3 odict_keys(['apple', 'banana', 'guava']) odict_values([30, 10, 20]) odict_items([('apple', 30), ('banana', 10), ('guava', 20)])
OrderedDict([('apple', 30), ('guava', 20)])


In [63]:
O = OrderedDict({"a":1, "b":3, "c":2})
O.popitem() # pop last
print(O)
O.popitem(last=False) # pop front
print(O)

OrderedDict([('a', 1), ('b', 3)])
OrderedDict([('b', 3)])


### SortedContainers

In [1]:
!pip install sortedcontainers

from sortedcontainers import SortedList 

In [2]:
T = SortedList([3,1,2,5,4])
T.add(6)
T.remove(1)
print(T) # [2,3,4,5,6]
print( T.index(0) )
# T.pop(index=-1) # same as list
# T.count(ele) # same as list
# T.islice(start,end) # generator

# tree based

SortedList([2, 3, 4, 5, 6])


ValueError: 0 is not in list